In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go

# Carguemos los datos

lugar de donde se consiguio la base de datos: https://github.com/jonyxnx/data_unam_examen_2024

In [3]:
datos = pd.read_json('data.json')

In [4]:
#veamos el tipo de datos que tenemos
datos.dtypes

Area         int64
Career      object
Faculty     object
Scores      object
Accepted      bool
dtype: object

In [5]:
#cambiemos el tipo de datos que tenemos
datos['Career'] = datos['Career'].astype('string')
datos['Faculty'] = datos['Faculty'].astype('string')
datos['Scores'] = pd.to_numeric(datos['Scores'], errors='coerce')
print(datos.dtypes)

Area                 int64
Career      string[python]
Faculty     string[python]
Scores             float64
Accepted              bool
dtype: object


In [6]:
#chequemos si tenemos valores faltantes
datos.isna().sum()

Area            0
Career          0
Faculty         0
Scores      23178
Accepted        0
dtype: int64

Tenemos que en scores hay 23178 datos faltantes, eso lo podriamos interpretar como la cantidad de aspirantes que se registraron pero no presentaron el examen

# Empecemos a hacer nuestro analisis

In [12]:
groped_data = datos.groupby('Area')
fig = go.Figure()

all_scores = pd.concat([group['Scores'] for _, group in groped_data])
overall_mean = all_scores.mean()

grouped_data_sorted = sorted(groped_data, key=lambda x: len(x[1]), reverse=True)

for area, group in grouped_data_sorted:
    fig.add_trace(go.Histogram(
        x=group['Scores'],
        name=f'Area {area}',
        opacity=0.7,
        marker=dict(
            line=dict(
                color='black',
                width=1
            )
        ),
        xbins=dict(
            start=min(group['Scores'].min(), 0),
            end=group['Scores'].max() + 10,
            size = 5
        )
    ))

fig.add_trace(go.Scatter(
        x=[overall_mean, overall_mean],
        y=[0,8000],
        mode='lines',
        line=dict(color='red', dash='dash'),
        name='Media de aciertos'
    ))

fig.update_layout(
    title='Distribución de Aciertos por Área',
    xaxis_title='Número de Aciertos',
    yaxis_title='Frecuencia',
    barmode='overlay',
    legend_title='Área',
    template='plotly_white'
)

fig.show()

* Podemos ver que el promedio de aciertos esta cercano a los 60.
* Las carreras de area 2 siguen siendo las mas demandadas.
* Las carreras de de area 4 son las menos demandadas.
* Una gran cantidad de aspirantes obtuvieron menos de 60 aciertos, sino es que la mayoria

In [14]:
promedio_aciertos = np.mean(datos['Scores'])
print(f'Podemos decir entonces que el promedio de de aciertos esta en {round(promedio_aciertos)} aciertos')

Podemos decir entonces que el promedio de de aciertos esta en 56 aciertos


Por curiosidad vamos a ver cuantas personas tuvieron un puntaje de 120

In [15]:
puntaje_maximos = 120
personas = 0
total_de_personas = 0
for puntaje in datos['Scores']:
    if puntaje == puntaje_maximos:
        total_de_personas = total_de_personas + 1

print(f'Hubo un total de {total_de_personas} de aspirantes con 120 aciertos')


Hubo un total de 6 de aspirantes con 120 aciertos


In [19]:
def calificacion(aciertos):
    cal = (aciertos * 100) / (puntaje_maximos * 10)
    return cal

print(f'Si calificamos en un rango de 0 a 10 entonces podriamos decir la calificacion promedio fue de {round(calificacion(promedio_aciertos),1)}')

Si calificamos en un rango de 0 a 10 entonces podriamos decir la calificacion promedio fue de 4.7


En que carreras se quedaron los aspirantes que obtuvieron las mayor cantidad de aciertos?

In [20]:
df_carreas_puntaje_maximo = datos[datos['Scores'] == puntaje_maximos]
df_carreas_puntaje_maximo.head(6)

,Area,Career,Faculty,Scores,Accepted
15041,1,INGENIERIA AEROESPACIAL,FACULTAD DE INGENIERÍA,120.0,True
17121,1,INGENIERIA CIVIL,FACULTAD DE INGENIERÍA,120.0,True
58790,2,MEDICINA VETERINARIA Y ZOOTECNIA,FACULTAD DE MEDICINA VETERINARIA Y ZOOTECNIA,120.0,True
67274,2,MEDICO CIRUJANO,FACULTAD DE MEDICINA,120.0,True
81620,2,MEDICO CIRUJANO,FES IZTACALA,120.0,True
104776,2,QUIMICA FARMACEUTICO BIOLOGICA,FACULTAD DE QUÍMICA,120.0,True


Veamos la cantidad de aspirantes que se quedaron en una carrera y tambien veamos cuantos fueron los rechazados (sin contar los que no presentaron examen)

In [43]:
numero_de_aspirantes_aceptados = 0
for aspirante in datos['Accepted']:
    if aspirante == True:
        numero_de_aspirantes_aceptados += 1

numero_de_aspirantes_rechazados = len(datos) - numero_de_aspirantes_aceptados
print(f'El numero de aspirantes que se quedaron en una carrera fue de {numero_de_aspirantes_aceptados} \n' +
      f'Mientras que el numero de aspirantes rechazados fue de {numero_de_aspirantes_rechazados}')

El numero de aspirantes que se quedaron en una carrera fue de 14000 
Mientras que el numero de aspirantes rechazados fue de 125285


Ahora veamoslo en porcentajes

In [54]:
print(f'porcentaje de aceptados: {round((numero_de_aspirantes_aceptados * 100) / len(datos), 2)}%')
print(f'porcentaje de rechazados: {round((numero_de_aspirantes_rechazados * 100) / len(datos), 2)}%')

porcentaje de aceptados: 10.05%
porcentaje de rechazados: 89.95%


Ahora veamos el top 10 de las carreras mas demandadas

In [21]:
# veamos cual fue la carrera mas demandada
demanda_carreras = datos['Career'].value_counts()
demanda_carreras_df = demanda_carreras.reset_index()
demanda_carreras_df.columns = ['Career', 'Counts']
demanda_carreras_df.head(10)

,Career,Counts
0,MEDICO CIRUJANO,25088
1,DERECHO,12119
2,PSICOLOGIA,10253
3,ARQUITECTURA,8473
4,CIRUJANO DENTISTA,8412
5,ADMINISTRACION,7652
6,CONTADURIA,6197
7,MEDICINA VETERINARIA Y ZOOTECNIA,6142
8,ENFERMERIA,6138
9,PEDAGOGIA,5136


Veamos el top 10 de las carreras menos demandadas

In [22]:
demanda_carreras_df.tail(10)

,Career,Counts
101,LENGUA Y LITERATURAS MODERNAS (LETRAS ITALIANAS),27
102,LENGUA Y LITERATURAS MODERNAS (LETRAS FRANCESAS),26
103,ESTUDIOS SOCIALES Y GESTION LOCAL,25
104,LITERATURA INTERCULTURAL,23
105,SOCIOLOGIA APLICADA,16
106,ETNOMUSICOLOGIA,14
107,GEOHISTORIA,8
108,DESARROLLO TERRITORIAL,8
109,LENGUA Y LITERATURAS MODERNAS (LETRAS ALEMANAS),7
110,LENGUA Y LITERATURAS MODERNAS (LETRAS PORTUGUE...,6


# Un analisis a los aspirantes a la facultad de ciencias

In [24]:
df_ciencias = datos[(datos['Faculty'] == 'FACULTAD DE CIENCIAS')]
numero_de_aceptados_ciencias = (df_ciencias['Accepted'] == True).sum()
numero_de_rechazados_en_ciencias = (df_ciencias['Accepted'] == False).sum()
print(f'El numero de alumnos que se quedaron en la Facultad de Ciencias es de {numero_de_aceptados_ciencias}')
print(f'El numero de aspirantes rechazados en ciencias fue de {numero_de_rechazados_en_ciencias}')

El numero de alumnos que se quedaron en la Facultad de Ciencias es de 486
El numero de aspirantes rechazados en ciencias fue de 5292


In [26]:
# organicemos la demanda de las carreras en ciencias
demanda_ciencias = df_ciencias['Career'].value_counts()
demanda_carreras_df = demanda_ciencias.reset_index()
demanda_ciencias.columns = ['Career', 'Counts']
demanda_ciencias.head(7)

Career
BIOLOGIA                      1529
ACTUARIA                      1214
CIENCIAS DE LA COMPUTACION    1043
FISICA                        1011
MATEMATICAS                    471
FISICA BIOMEDICA               319
MATEMATICAS APLICADAS          191
Name: count, dtype: Int64